In [1]:
import numpy as np, pandas as pd
import train as t
from src import helper_models as hm, helper_data as hd, helper_tts as htts
from importlib import reload
import keras, glob




Using TensorFlow backend.


In [ ]:
# Reload imports of custom modules, in case I am modifying and reloading them live
reload(t)
reload(hd)
reload(hm)
reload(htts)


# Initialize simulation

In [2]:
sim=t.Ctrain()

# Init parameters
Parameters must always be updated through the `UpdateParams()` method

In [3]:
sim.UpdateParams(
    datapaths=['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/',
               './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/'],
    outpath='out_example', 
    L=128, 
    model='conv2', 
    aug=True, 
    datakind='mixed', 
    ttkind='mixed',
    class_select=['chaoborus','bosmina','unknown_plankton'] #None
    )


Create output directory only after you've set the right `outpath`

In [4]:
sim.CreateOutDir()


In [5]:
print(sim.params.datapaths)


['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/', './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/']


# Load data

In [6]:
# These arguments are the defaults
reload(hd)
sim.LoadData(L=sim.params.L, class_select=sim.params.class_select, datakind=sim.params.datakind) 

datapaths: ['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/', './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/']
classes from datapaths: ['hydra', 'dirt', 'unknown_plankton', 'fragilaria', 'nauplius', 'paradileptus', 'copepod_skins', 'fish', 'eudiaptomus', 'daphnia', 'filament', 'conochilus', 'cyclops', 'diatom_chain', 'keratella_cochlearis', 'unknown', 'aphanizomenon', 'maybe_cyano', 'diaphanosoma', 'kellikottia', 'bosmina', 'leptodora', 'keratella_quadrata', 'rotifers', 'uroglena', 'synchaeta', 'chaoborus', 'asterionella', 'ceratium', 'asplanchna', 'trichocerca', 'daphnia_skins', 'dinobryon', 'polyarthra']
class: chaoborus (10)
class: bosmina (80)
class: unknown_plankton (71)


# Create sets (test and train)

In [7]:
sim.CreateTrainTestSets(ttkind=sim.params.ttkind)


# Create and Train Model
Custom parameter changes are made by acting directly on the params class through the `UpdateParams()` method.

In [8]:
sim.UpdateParams(totEpochs=1000)
sim.Train()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/1000
4/4 [==============================] - 0s 85ms/step - loss: 1.1965 - accuracy: 0.4766 - val_loss: 1.1316 - val_accuracy: 0.5758
Epoch 2/1000
4/4 [==============================] - 0s 41ms/step - loss: 1.1936 - accuracy: 0.4766 - val_loss: 1.1290 - val_accuracy: 0.5758
Epoch 3/1000
4/4 [==============================] - 0s 58ms/step - loss: 1.1913 - accuracy: 0.4766 - val_loss: 1.1265 - val_accuracy: 0.5758
Epoch 4/1000
4/4 [==============================] - 0s 71ms/step - loss: 1.1882 - accuracy: 0.4766 - val_loss: 1.1240 - val_accuracy: 0.5758
Epoch 5/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.1857 - accuracy: 0.4766 - val_loss: 1.1215 - val_accuracy: 0.5758
Epoch 6/1000
4/4 [==============================] - 0s 70ms/step - loss: 1.1829 - accuracy: 0.4766 - val_loss: 1.1191 - val_accuracy: 0.5758
Epoch 7/1000
4/4 [==============================] - 0s 68ms/step - loss:

Epoch 55/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.0801 - accuracy: 0.4766 - val_loss: 1.0231 - val_accuracy: 0.5758
Epoch 56/1000
4/4 [==============================] - 0s 70ms/step - loss: 1.0784 - accuracy: 0.4766 - val_loss: 1.0216 - val_accuracy: 0.5758
Epoch 57/1000
4/4 [==============================] - 0s 67ms/step - loss: 1.0768 - accuracy: 0.4766 - val_loss: 1.0201 - val_accuracy: 0.5758
Epoch 58/1000
4/4 [==============================] - 0s 71ms/step - loss: 1.0751 - accuracy: 0.4766 - val_loss: 1.0186 - val_accuracy: 0.5758
Epoch 59/1000
4/4 [==============================] - 0s 69ms/step - loss: 1.0738 - accuracy: 0.4766 - val_loss: 1.0172 - val_accuracy: 0.5758
Epoch 60/1000
4/4 [==============================] - 0s 71ms/step - loss: 1.0722 - accuracy: 0.4766 - val_loss: 1.0157 - val_accuracy: 0.5758
Epoch 61/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.0706 - accuracy: 0.4766 - val_loss: 1.0143 - val_accuracy: 0.5758
Epoch 

4/4 [==============================] - 0s 71ms/step - loss: 1.0090 - accuracy: 0.4766 - val_loss: 0.9578 - val_accuracy: 0.5758
Epoch 113/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.0080 - accuracy: 0.4766 - val_loss: 0.9570 - val_accuracy: 0.5758
Epoch 114/1000
4/4 [==============================] - 0s 72ms/step - loss: 1.0071 - accuracy: 0.4766 - val_loss: 0.9562 - val_accuracy: 0.5758
Epoch 115/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.0062 - accuracy: 0.4766 - val_loss: 0.9553 - val_accuracy: 0.5758
Epoch 116/1000
4/4 [==============================] - 0s 72ms/step - loss: 1.0054 - accuracy: 0.4766 - val_loss: 0.9545 - val_accuracy: 0.5758
Epoch 117/1000
4/4 [==============================] - 0s 69ms/step - loss: 1.0045 - accuracy: 0.4766 - val_loss: 0.9538 - val_accuracy: 0.5758
Epoch 118/1000
4/4 [==============================] - 0s 71ms/step - loss: 1.0037 - accuracy: 0.4766 - val_loss: 0.9530 - val_accuracy: 0.5758
Epoch 119/1000

4/4 [==============================] - 0s 72ms/step - loss: 0.9700 - accuracy: 0.4766 - val_loss: 0.9214 - val_accuracy: 0.5758
Epoch 170/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.9696 - accuracy: 0.4766 - val_loss: 0.9210 - val_accuracy: 0.5758
Epoch 171/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.9691 - accuracy: 0.4766 - val_loss: 0.9205 - val_accuracy: 0.5758
Epoch 172/1000
4/4 [==============================] - 0s 66ms/step - loss: 0.9685 - accuracy: 0.4766 - val_loss: 0.9200 - val_accuracy: 0.5758
Epoch 173/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.9681 - accuracy: 0.4766 - val_loss: 0.9195 - val_accuracy: 0.5758
Epoch 174/1000
4/4 [==============================] - 0s 68ms/step - loss: 0.9679 - accuracy: 0.4766 - val_loss: 0.9190 - val_accuracy: 0.5758
Epoch 175/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.9672 - accuracy: 0.4766 - val_loss: 0.9186 - val_accuracy: 0.5758
Epoch 176/1000

4/4 [==============================] - 0s 63ms/step - loss: 0.9471 - accuracy: 0.4766 - val_loss: 0.8989 - val_accuracy: 0.5758
Epoch 227/1000
4/4 [==============================] - 0s 70ms/step - loss: 0.9468 - accuracy: 0.4766 - val_loss: 0.8986 - val_accuracy: 0.5758
Epoch 228/1000
4/4 [==============================] - 0s 63ms/step - loss: 0.9464 - accuracy: 0.4766 - val_loss: 0.8983 - val_accuracy: 0.5758
Epoch 229/1000
4/4 [==============================] - 0s 69ms/step - loss: 0.9461 - accuracy: 0.4766 - val_loss: 0.8980 - val_accuracy: 0.5758
Epoch 230/1000
4/4 [==============================] - 0s 64ms/step - loss: 0.9460 - accuracy: 0.4766 - val_loss: 0.8977 - val_accuracy: 0.5758
Epoch 231/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.9455 - accuracy: 0.4766 - val_loss: 0.8974 - val_accuracy: 0.5758
Epoch 232/1000
4/4 [==============================] - 0s 63ms/step - loss: 0.9453 - accuracy: 0.4766 - val_loss: 0.8971 - val_accuracy: 0.5758
Epoch 233/1000

...

4/4 [==============================] - 0s 67ms/step - loss: 0.9021 - accuracy: 0.4766 - val_loss: 0.8393 - val_accuracy: 0.5758
Epoch 740/1000
4/4 [==============================] - 0s 70ms/step - loss: 0.9018 - accuracy: 0.4766 - val_loss: 0.8393 - val_accuracy: 0.5758
Epoch 741/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.9019 - accuracy: 0.4766 - val_loss: 0.8392 - val_accuracy: 0.5758
Epoch 742/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.9019 - accuracy: 0.4766 - val_loss: 0.8392 - val_accuracy: 0.5758
Epoch 743/1000
4/4 [==============================] - 0s 68ms/step - loss: 0.9018 - accuracy: 0.4766 - val_loss: 0.8392 - val_accuracy: 0.5758
Epoch 744/1000
4/4 [==============================] - 0s 70ms/step - loss: 0.9015 - accuracy: 0.4766 - val_loss: 0.8391 - val_accuracy: 0.5758
Epoch 745/1000
4/4 [==============================] - 0s 66ms/step - loss: 0.9020 - accuracy: 0.4766 - val_loss: 0.8391 - val_accuracy: 0.5758
Epoch 746/1000

4/4 [==============================] - 0s 71ms/step - loss: 0.9010 - accuracy: 0.4844 - val_loss: 0.8371 - val_accuracy: 0.5758
Epoch 797/1000
4/4 [==============================] - 0s 68ms/step - loss: 0.9010 - accuracy: 0.4844 - val_loss: 0.8371 - val_accuracy: 0.5758
Epoch 798/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.9007 - accuracy: 0.4766 - val_loss: 0.8371 - val_accuracy: 0.5758
Epoch 799/1000
4/4 [==============================] - 0s 68ms/step - loss: 0.9008 - accuracy: 0.4844 - val_loss: 0.8370 - val_accuracy: 0.5758
Epoch 800/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.9009 - accuracy: 0.4766 - val_loss: 0.8370 - val_accuracy: 0.5758
Epoch 801/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.9010 - accuracy: 0.4766 - val_loss: 0.8370 - val_accuracy: 0.5758
Epoch 802/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.9009 - accuracy: 0.4922 - val_loss: 0.8369 - val_accuracy: 0.5758
Epoch 803/1000

4/4 [==============================] - 0s 68ms/step - loss: 0.9002 - accuracy: 0.4766 - val_loss: 0.8353 - val_accuracy: 0.5758
Epoch 854/1000
4/4 [==============================] - 0s 70ms/step - loss: 0.9005 - accuracy: 0.4844 - val_loss: 0.8352 - val_accuracy: 0.5758
Epoch 855/1000
4/4 [==============================] - 0s 68ms/step - loss: 0.9006 - accuracy: 0.4844 - val_loss: 0.8352 - val_accuracy: 0.5758
Epoch 856/1000
4/4 [==============================] - 0s 70ms/step - loss: 0.9002 - accuracy: 0.4844 - val_loss: 0.8352 - val_accuracy: 0.5758
Epoch 857/1000
4/4 [==============================] - 0s 69ms/step - loss: 0.9002 - accuracy: 0.4844 - val_loss: 0.8352 - val_accuracy: 0.5758
Epoch 858/1000
4/4 [==============================] - 0s 69ms/step - loss: 0.9001 - accuracy: 0.4844 - val_loss: 0.8351 - val_accuracy: 0.5758
Epoch 859/1000
4/4 [==============================] - 0s 64ms/step - loss: 0.9002 - accuracy: 0.4844 - val_loss: 0.8351 - val_accuracy: 0.5758
Epoch 860/1000

4/4 [==============================] - 0s 68ms/step - loss: 0.8996 - accuracy: 0.4922 - val_loss: 0.8336 - val_accuracy: 0.5758
Epoch 911/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.8997 - accuracy: 0.4766 - val_loss: 0.8336 - val_accuracy: 0.5758
Epoch 912/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.8999 - accuracy: 0.4844 - val_loss: 0.8336 - val_accuracy: 0.5758
Epoch 913/1000
4/4 [==============================] - 0s 70ms/step - loss: 0.8997 - accuracy: 0.4766 - val_loss: 0.8336 - val_accuracy: 0.5758
Epoch 914/1000
4/4 [==============================] - 0s 68ms/step - loss: 0.8996 - accuracy: 0.4766 - val_loss: 0.8335 - val_accuracy: 0.5758
Epoch 915/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.8996 - accuracy: 0.4844 - val_loss: 0.8335 - val_accuracy: 0.5758
Epoch 916/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.8999 - accuracy: 0.4766 - val_loss: 0.8335 - val_accuracy: 0.5758
Epoch 917/1000

4/4 [==============================] - 0s 67ms/step - loss: 0.8992 - accuracy: 0.4844 - val_loss: 0.8322 - val_accuracy: 0.5758
Epoch 968/1000
4/4 [==============================] - 0s 70ms/step - loss: 0.8992 - accuracy: 0.4844 - val_loss: 0.8322 - val_accuracy: 0.5758
Epoch 969/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.8992 - accuracy: 0.4844 - val_loss: 0.8322 - val_accuracy: 0.5758
Epoch 970/1000
4/4 [==============================] - 0s 70ms/step - loss: 0.8994 - accuracy: 0.4766 - val_loss: 0.8321 - val_accuracy: 0.5758
Epoch 971/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.8989 - accuracy: 0.4844 - val_loss: 0.8321 - val_accuracy: 0.5758
Epoch 972/1000
4/4 [==============================] - 0s 69ms/step - loss: 0.8993 - accuracy: 0.4766 - val_loss: 0.8321 - val_accuracy: 0.5758
Epoch 973/1000
4/4 [==============================] - 0s 64ms/step - loss: 0.8991 - accuracy: 0.4766 - val_loss: 0.8321 - val_accuracy: 0.5758
Epoch 974/1000

# Classification Report

In [9]:
sim.Report()

                  precision    recall  f1-score   support

         bosmina       0.58      1.00      0.73        19
       chaoborus       0.00      0.00      0.00         1
unknown_plankton       0.00      0.00      0.00        13

        accuracy                           0.58        33
       macro avg       0.19      0.33      0.24        33
    weighted avg       0.33      0.58      0.42        33



/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


We now extract manually the training loss corresponding to the best weights, so that we can make sure that restarting the simulation works

In [ ]:
def BestLoss(history):
    ''' Returns the training loss of the point where the validation loss was minimal'''
    return history['loss'][np.argmin(history['val_loss'])]
def InitLoss(history):
    ''' Returns the training loss of the point where the validation loss was minimal'''
    return history['loss'][0]

initLoss = InitLoss(sim.history.history)
bestLoss = BestLoss(sim.history.history)

print('Initial training loss:',initLoss)
print('Best    training loss:',bestLoss)

# Load Best Weights and keep training
Now we load the best weights, just to show that the loss does not start from scratch again.
If `sim.params.load` is set to `None`, the simulation starts from scratch. Otherwise, it can be set to the path to saved weights (as in this example).
I also show how to play around with some input parameters.

In the current setting, we expect that, if the learning rate is not too big, the loss should still decrease with respect to BestLoss.

In [ ]:
# Load the weights
sim.params.load=sim.params.outpath+'/bestweights.hdf5'

# Set initial epoch to the end of the previous run, and extend the total number of epochs (otherwise it won't run)
sim.params.initial_epoch=sim.history.epoch[-1]+1 if len(sim.history.epoch)>0 else 0
sim.params.totEpochs=sim.params.initial_epoch+5

print('Epochs for the first run:', sim.history.epoch)
sim.Train()
print('Epochs for the second run:', sim.history.epoch)


Compare the initial loss with the best loss of the previous run. The new one should be smaller or equal, since we expect the loss to be decreasing.

In [ ]:
# Compare checkpointed loss with loss at the very beginning of the simulation
print('This quantity should be negative in normal conditions:',bestLoss-initLoss)

# Compare checkpointed loss with loss at the beginning of the run after the checkpoint (i.e. one epoch later)
print('This quantity should be non negative in normal conditions:',bestLoss-InitLoss(sim.history.history))

sim.Report()

# Start Other Simulation from Scratch

Load model, parameters and classes that were generated in the previous simulation.

Of course, we could just get them from `sim`, but here we are simulating that this is an entirely independent run

In [ ]:
model_from_previous=sim.params.outpath+'/'+sim.params.saveModelName
params_from_previous, classes_from_previous = hd.ReadArgsTxt(sim.params.outpath)

We initialize a new simulation, but load everything from the files that were generated by the previous one.

In [ ]:

sim2=t.Ctrain()


sim2.UpdateParams(
                outpath='out_example2',
                L=params_from_previous['L'],
                model=params_from_previous['model'],
                datapath=params_from_previous['datapath'],
                datakind=params_from_previous['datakind'],
                ttkind=params_from_previous['ttkind'],
                class_select=classes_from_previous
                )

sim2.CreateOutDir()
sim2.LoadData() # Should make a deep copy from sim
sim2.CreateTrainTestSets() # Should make a deep copy from sim
sim2.LoadModel(model_from_previous)





In [ ]:
# The classification report should coincide with the one at the end of simulation 1
sim2.Report()
# sim2.Predict()

# Load Best Weights
We load the best weights from the callback, and make the classification report again. We expect slightly better results.

In [ ]:
# BISOGNA FARE LOADING DEI BEST WEIGHTS
sim.model.load_weights(sim.params.outpath+'/bestweights.hdf5')
# !ls out_example/
sim.Report()

# Use second model to predict on new, unseen data

In [ ]:
testdir = 'data/1_zooplankton_0p5x/validation/tommy_validation/images/dinobryon/'
im_names=np.array(glob.glob(testdir+'/*.jpeg'),dtype=object)
npimages=hd.LoadImageList(im_names, L=sim2.params.L, show=False)

probs=sim2.model.predict(npimages)
predictions=probs.argmax(axis=1)  # The class that the classifier would bet on


In [ ]:
print('Predictions:',sim2.tt.lb.classes_[predictions])